# ch03. word2vec
'추론 기반 학습' : 추론 (word2vec)

## 3.1 추론 기반 기법과 신경망
단어를 벡터로 표현하는 방법
- 통계 기반 기법
- 추론 기반 기법

### 3.1.1 통계 기반 기법의 문제점
통계 기반 기법 : 주변 단어의 빈도를 기초로 단어 표현
- 단어의 동시 발생 행렬
- 그 행렬에 SVD 적용
- 밀집벡터 표현을 얻음
- 하지만 이 방식은 대규모 말뭉치를 담을 때 문제 발생함

=> 말뭉치 전체의 통계를 이용해 **단 1회의 처리**만에 단어의 분산 표현을 얻음

추론 기반 기법: 신경망을 이용하는 경우는 **미니배치**로 학습하는 것이 일반적
- 미니배치 학습: 신경망이 한 번에 소량의 학습 샘플씩 반복해서 학습하며 가중치를 갱신해 감

![통계 기반 기법과 추론 기반 기법 비교](img/3-1.png)

[통계 기반 기법]
- 학습 데이터를 한꺼번에 처리 (배치 학습)

[추론 기반 기법]
- 학습 데이터의 일부를 사용하여 순차적으로 학습 (미니배치 학습)
- 여러 머신과 여러 GPU를 이용한 병렬 계산도 가능
- 학습 속도를 높일 수 있음

### 3.1.2 추론 기반 기법 개요
추론: 주변 단어(맥락)이 주어졌을 때 "?" 안에 무슨 단어가 들어가는지를 추측하는 작업
![](img/3-2.png)

=> 이러한 추론 문제를 반복해 풀면서 단어의 출현 패턴을 학습하는 것
![](img/3-3.png)

* 모델: 맥락 정보를 입력받아 (출현할 수 있는) 각 단어의 출현 확률 출력
* 이러한 틀 안에서 말뭉치를 사용해 모델이 올바른 추측을 할 수 있도록
* 그 학습의 결과로 단어의 분산 표현을 얻는 것이 추론 기반 기법의 전체 그림

### 3.1.3 신경망에서의 단어 처리
신경망: "you", "say"와 은 단어를 있는 그대로 처리X -> 단어를 _고정 길이의 벡터_ 로 변환해야.
- 이 때 사용하는 방법: 원핫 표현 (원핫 벡터)

-> 벡터의 원소 중 하나만 1이고 나머지는 모두 0인 벡터

1. 총 어휘 수만큼의 원소를 갖는 벡터 준비
2. 인덱스가 단어 ID와 같은 원소를 1로, 나머지는 모두 0으로 설정
-> 단어를 고정 길이 벡터로 변환하면 우리 신경망의 입력층은 뉴런의 수를 **고정** 할 수 있음

![](img/3-5.png)

=> 단어를 벡터로 나타낼 수 있고, 신경망을 구성하는 계층들은 벡터를 처리할 수 있음

![](img/3-7.png)

In [1]:
import numpy as np

c = np.array([[1, 0, 0, 0, 0, 0, 0]])   # 입력
W = np.random.randn(7, 3)               # 가중치
h = np.matmul(c, W)                     # 중간 노
print(h)

[[ 1.15399587 -0.39786194  0.26368861]]


단어 ID가 0인 단어를 원핫 표현으로 표현한 다음 완전연결계층 (matmul)

## 3.2 단순한 word2vec
=> CBOW(continous bag-of-words) 모델 사용

### 3.2.1 CBOW 모델의 추론 처리


In [2]:
import sys
sys.path.append("..")
import numpy as np
from common.layers import MatMul

c0 = np.array([[1, 0, 0, 0, 0, 0, 0]])
c1 = np.array([[0, 0, 1, 0, 0, 0, 0]])

# 가중치 초기
W_in = np.random.randn(7, 3)
W_out = np.random.randn(3, 7)

# 계층 생성
in_layer0 = MatMul(W_in)
in_layer1 = MatMul(W_in)
out_layer = MatMul(W_out)

# 순전파
h0 = in_layer0.forward(c0)
h1 = in_layer1.forward(c1)
h = 0.5 * (h0 + h1)
s = out_layer.forward(h)

print(s)

[[ 0.68351986 -0.51137835 -1.23924066 -1.20130729 -1.13300778 -0.956684
  -2.8252252 ]]


### 3.2.2 CBOW 모델의 학습
CBOW 모델: 출력층에서 각 단어의 점수 출력

-> softmax 적용: '확률'을 얻을 수 있음 (맥락이 주어졌을 때 그 중앙에 어떤 단어가 출현하는지)


- 소프트맥스 함수를 이용해 점수를 확률으로 변환
- 확률을 정답 레이블로부터 교차 엔트로피 오차를 구한 후
- 그 값을 손실로 사용해 학습 진행
![](img/3-14.png)


### 3.2.3 word2vec의 가중치와 분산 표현
<word2vec에서 사용하는 가중치>
- 입력 측 완전연결계층의 가중치 (w_in)
- 출력 측 완전연결계층의 가중치 (w_out)

<각 단어의 분산 표현>
- 입력 측 가중치 W_in의 행
- 출력 측 가중치 W_out의 열

<최종적으로 이용하는 단어의 분산 표현으로는...?>
**1. 입력 측의 가중치만 이용한다**
2. 출력 측의 가중치만 이용한다
3. 양쪽 가중치를 모두 이용한다

> 많은 연구에서 출력 측 가중치는 버리고 입력 측 가중치 (W_in)만을
> 최종 단어의 분산 표현으로 이용함

## 3.3 학습 데이터 준비
### 3.3.1 맥락과 타깃


In [6]:
import sys
import numpy as np
sys.path.append("..")
from common.util import preprocess

text = "You say goodbye and I say hello."
corpus, word_to_id, id_to_word = preprocess(text)
print(corpus)

print(id_to_word)

[0 1 2 3 4 1 5 6]
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


In [10]:
# 맥락과 타깃을 만드는 함수
def create_contexts_target(corpus, window_size=1):
    target = corpus[window_size:-window_size]
    contexts = []

    for idx in range(window_size, len(corpus)-window_size):
        cs = []
        for t in range(-window_size, window_size+1):
            if t == 0: continue
            cs.append(corpus[idx+t])
        contexts.append(cs)

    return np.array(contexts), np.array(target)

In [11]:
contexts, target = create_contexts_target(corpus, window_size=1)
print(contexts)
print(target)

[[0 2]
 [1 3]
 [2 4]
 [3 1]
 [4 5]
 [1 6]]
[1 2 3 4 1 5]


### 3.3.2 원핫 표현으로 변환
맥락 & 타깃 -> 단어 ID에서 원핫 표현으로 변환

**다차원 배열의 형상** 에 주목해야 함

In [12]:
import sys
sys.path.append("..")
from common.util import preprocess, create_contexts_target, convert_one_hot

text = "You say goodbye and I say hello."
corpus, word_to_id, id_to_word = preprocess(text)

contexts, target = create_contexts_target(corpus, window_size=1)

vocab_size = len(word_to_id)
target = convert_one_hot(target, vocab_size)
contexts = convert_one_hot(contexts, vocab_size)

## 3.4 CBOW 모델 구현
이번 장에는 간단한 cbow 모델 구현

In [13]:
import sys
sys.path.append("..")
import numpy as np
from common.layers import MatMul, SoftmaxWithLoss

class SimpleCBOW:
    def __init__(self, vocab_size, hidden_size):
        V, H = vocab_size, hidden_size

        # 가중치 초기화
        W_in = 0.01 * np.random.rand(V, H).astype('float32')
        W_out = 0.01 * np.random.rand(V, H).astype('float32')

        # 계층 생성
        self.in_layer0 = MatMul(W_in)
        self.in_layer1 = MatMul(W_in)
        self.out_layer = MatMul(W_out)
        self.loss_layer = SoftmaxWithLoss()

        # 모든 가중치와 기울기를 리스트에 모음
        layers = [self.in_layer0, self.in_layer1, self.out_layer]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        self.word_vecs = W_in


    def forward(self, context, target):
        h0 = self.in_layer0.forward(context[:, 0])
        h1 = self.in_layer0.forward(context[:, 1])
        h = 0.5 * (h0 + h1)
        score = self.out_layer.forward(h)
        loss = self.loss_layer.forward(score, target)
        return loss

    def backward(self, dout=1):
        ds = self.loss_layer.backward(dout)
        da = self.out_layer(ds)
        da *= 0.5
        self.in_layer1.backward(da)
        self.in_layer0.backward(da)
        return None

In [15]:
import sys
sys.path.append("..")
from common.trainer import Trainer
from common.optimizer import Adam
from common.util import preprocess, create_contexts_target, convert_one_hot

window_size = 1
hidden_size = 5
batch_size = 3
max_epochs = 1000

text = "You say goodbye and I say hello."
corpus, id_to_word, word_to_id = preprocess(text)

vocab_size = len(word_to_id)
contexts, target = create_contexts_target(corpus, window_size=1)
target = convert_one_hot(target, vocab_size)
contexts = convert_one_hot(contexts, vocab_size)

model = SimpleCBOW(vocab_size, hidden_size)
optimizer = Adam(model.params, lr=0.001)
trainer = Trainer(model, optimizer)

trainer.fit(contexts, target, max_epochs, batch_size)
trainer.plot()

ImportError: dlopen(/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/PIL/_imaging.cpython-310-darwin.so, 0x0002): symbol not found in flat namespace '_jpeg_resync_to_restart'

## 3.5 word2vec 보충
- CBOW 모델: 맥락이 여러 개, 그 맥락들로부터 중앙의 단어 (타깃) 추측
- skip-gram 모델: 중앙의 단어로부터 주변의 여러 단어(맥락)을 추측

=> 말뭉치가 커질 수록 저빈도 단어나 유추 문제의 성능 면에서 skip-gram모델이 더 뛰어남

### 3.5.3. 통계 기반 vs. 추론 기반
- 통계 기반: 말뭉치의 전체 통계로부터 1회 학습하여 단어의 분산 표현을 얻음
    - 새 단어가 생기면? 계산을 처음부터 다시. (동시발생행렬 -> SVD)
    - 단어의 유사성이 인코딩됨
- 추론 기반: 말뭉치를 일부분씩 여러 번 보면서 학습 (미니배치)
    - 매개변수 다시 학습 가능
    - 기존에 학습한 경험을 해치지 않으면서 단어의 분산 표현을 효율적으로 갱신할 수 있음
    - 복잡한 단어 사이의 패턴까지도 파악되어 인코딩됨.


## 3.6 정리
- CBOW : 기본적으로2 층 구성의 아주 단순한 신경망

---

### 이번 장에서 배운 내용
- 추론 기반 기법은 추측하는 것이 목적이며, 그 부산물로 단어의 분산 표현을 얻을 수 있다.
- word2vec은 추론 기반 기법이며, 단순한 2층 신경망이다
- word2vec은 skip-gram 모델과 CBOW 모델을 제공한다.
- CBOW 모델은 여러 단어로부터 하나의 단어를 추측한다
- skip-gram 모델은 하나의 단어로부터 다수의 단어를 추측한다.
- word2vec은 가중치를 다시 학습할 수 있으므로, 단어의 분산 표현이나 새로운 단어 추각를 효율적으로 수행할 수 있다.   